In [1]:
import sys

sys.path.insert(1, "../utils/")

# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [ ]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [ ]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [2]:
from typing import Union, Callable, Optional, Any, Dict
from numbers import Number
import my_math as math
from dataclasses import dataclass
import random


@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(float(other) - self.value, self.d)

    def __rsub__(self, other: Number) -> "Dual":
        assert isinstance(other, Number)

        return Dual(float(other) - self.value, -self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    __rmul__ = __mul__
    __radd__ = __add__

    def __pos__(self) -> "Dual":
        return Dual(self.value, self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)

    def __truediv__(self, other: Union["Dual", Number]):
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (o_value * self.d - self.value * o_d) / o_value ** 2)
            case Number():
                return Dual(self.value / float(other), self.d / float(other))

    def __rtruediv__(self, other: Number) -> "Dual":
        assert isinstance(other, Number)

        return Dual(float(other) / self.value, -float(other) * self.d / self.value ** 2)

    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        if self.value == 0:
            return Dual(0., 0.)
        match other:
            case Dual(o_value, o_d):
                if o_value != 0:
                    temp1 = self.value ** (other.value - 1)
                    temp2 = temp1 * self.value
                    return Dual(temp2, self.d * o_value * temp1 + o_d * temp2 * math.log(self.value))
                else:
                    return Dual(1, o_d * self.value)
            case Number():
                if other == 0:
                    return Dual(1., 0.)
                other = float(other)
                temp = self.value ** (other - 1)
                return Dual(temp * self.value, other * temp * self.d)

    def __rpow__(self, other: Number) -> "Dual":
        assert isinstance(other, Number)

        return Dual(temp := float(other) ** self.value, temp * self.d * math.log(other))

    def __abs__(self) -> "Dual":
        assert self.value != 0
        return Dual(abs(self.value), self.d if self.value > 0 else -self.d)

    def __str__(self) -> str:
        return f"{self.value:.3f}" + (f"+{self.d}ε" if self.d else "")


def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

In [3]:
import unittest
import random


class TestAutomaticDifferential(unittest.TestCase):
    def setUp(self):
        self.const = lambda c: Dual(c, 0)

    def test_arithmetic(self):
        f1 = lambda x: x + x ** 2 - 5
        f2 = lambda x: x - 1/x
        f3 = lambda x: x ** 3 / x ** 7
        f4 = lambda x: x ** 6 + (x - 1/x) ** 3 + (x + x ** 2) / (x ** 2 - x)
        f5 = lambda x: abs(-x + 6) - abs(x ** 3 - x)
        f6 = lambda x: 5 ** (x + 7 - 1 / x)
        f7 = lambda x: x ** x ** 2
        f8 = lambda x: self.const(7)
        
        for x in range(-10, 11):
            assert math.isclose(diff(f1)(x=x), 1 + 2 * x)
            if x != 0:
                assert math.isclose(diff(f2)(x=x), 1 + 1 / x ** 2)
                assert math.isclose(diff(f3)(x=x), -4 / x ** 5)
                if x != 1:
                    assert math.isclose(diff(f4)(x=x), 6 * x ** 5 + 3 * (x - 1 / x) ** 2 * (1 + 1/x**2) - 2  / (x - 1) ** 2)
                if x != 6 and x != 1 and x != -1:
                    assert math.isclose(diff(f5)(x=x), -math.sgn(-x + 6) - (3 * x ** 2 - 1) * math.sgn(x ** 3 - x))
                assert math.isclose(diff(f6)(x=x), 5 ** (x + 7 - 1/x) * math.log(5) * (1 + 1/x**2))
            if x > 0:
                assert math.isclose(diff(f7)(x=x), x ** (x ** 2 + 1) * (2 * math.log(x) + 1))
            assert math.isclose(diff(f8)(x=x), 0, abs_tol=1e-9)

        random.seed(42)
        for x in [-10 + 20 * random.random() for _ in range(100)]:
            assert math.isclose(diff(f1)(x=x), 1 + 2 * x)
            if x != 0:
                assert math.isclose(diff(f2)(x=x), 1 + 1 / x ** 2)
                assert math.isclose(diff(f3)(x=x), -4 / x ** 5)
                if x != 1:
                    assert math.isclose(diff(f4)(x=x), 6 * x ** 5 + 3 * (x - 1 / x) ** 2 * (1 + 1/x**2) - 2  / (x - 1) ** 2)
                if x != 6 and x != 1 and x != -1:
                    assert math.isclose(diff(f5)(x=x), -math.sgn(-x + 6) - (3 * x ** 2 - 1) * math.sgn(x ** 3 - x))
                assert math.isclose(diff(f6)(x=x), 5 ** (x + 7 - 1/x) * math.log(5) * (1 + 1/x**2))
            if x > 0:
                assert math.isclose(diff(f7)(x=x), x ** (x ** 2 + 1) * (2 * math.log(x) + 1))
            assert math.isclose(diff(f8)(x=x), 0, abs_tol=1e-9)


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.017s

OK


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [4]:
def exp(x: Dual) -> Dual:
    return Dual(temp := math.exp(x.value), temp * x.d)


def log(x: Union[Dual, Number], a: Optional[Union[Number, Dual]] = None) -> Dual:
    if a is None:
        match x:
            case Dual(value, d):
                return Dual(math.log(value), d / value)
            case Number:
                return Dual(math.log(x), 0)
    else:
        return log(x) / log(a)


def sin(x: Dual) -> Dual:
    return Dual(math.sin(x.value), x.d * math.cos(x.value))


def cos(x: Dual) -> Dual:
    return Dual(math.cos(x.value), x.d * -math.sin(x.value))


def tan(x: Dual) -> Dual:
    assert abs((x.value % math.pi) - math.pi / 2) > 1e-9
    return sin(x) / cos(x)


def cotan(x: Dual) -> Dual:
    assert abs(x.value % math.pi) > 1e-9
    return cos(x) / sin(x)


In [5]:
import unittest
import random


class TestAutomaticDifferential(unittest.TestCase):
    def setUp(self):
        self.const = lambda c: Dual(c, 0)

    def test_arithmetic(self):
        f1 = lambda x: x * exp(x)
        f2 = lambda x: cos(x + math.pi / 2) + sin(x)
        f3 = lambda x: cos(x) ** 2 / (tan(x) + cotan(x))
        f4 = lambda x: sin(self.const(2.31)) ** log(x)
        f5 = lambda x: sin(x) * cos(x**2) - exp(x)
        f6 = lambda x: exp(log(x))
        f7 = lambda x: log(10 + sin(5*x - 2/x**2))
        f8 = lambda x: sin(cos(x)) + tan(x) / 5 + exp(x ** 0.1 * sin(x)) / log(x)
        f9 = lambda x: log(x, x)
        f10 = lambda x: log(abs(tan(x)), 2 ** x)
        
        random.seed(42)
        for x in [-10 + 20 * random.random() for _ in range(100)]:
            assert math.isclose(diff(f1)(x=x), (x + 1) * math.exp(x))
            assert math.isclose(diff(f2)(x=x), 0, abs_tol=1e-9)
            assert math.isclose(diff(f3)(x=x), -3 * math.cos(x) ** 2 * math.sin(x) ** 2 + math.cos(x) ** 4)
            if x > 0:
                assert math.isclose(diff(f4)(x=x), math.sin(2.31) ** math.log(x) * math.log(math.sin(2.31)) / x)
            assert math.isclose(diff(f5)(x=x), math.cos(x) * math.cos(x ** 2) - 2 * x * math.sin(x) * math.sin(x ** 2) - math.exp(x))
            if x > 0:
                assert math.isclose(diff(f6)(x=x), 1)
            assert math.isclose(diff(f7)(x=x), math.cos(5 * x - 2 / x**2) * (5 + 4 / x**3) / (10 + math.sin(5 * x - 2 / x**2)))
            if x > 0:
                assert math.isclose(diff(f8)(x=x), -math.cos(math.cos(x)) * math.sin(x) + 1 / (5 * math.cos(x) ** 2) + math.exp(x ** 0.1 * math.sin(x)) * (math.log(x) * (0.1 * x ** -0.9 * math.sin(x) + x ** 0.1 * math.cos(x)) - 1 / x) / math.log(x) ** 2)
                assert math.isclose(diff(f9)(x=x), 0, abs_tol=1e-9)
            assert math.isclose(diff(f10)(x=x), ((2 * x * math.log(2)) / (math.sin(2 * x)) - math.log(2) * math.log(abs(math.tan(x)))) / ((x * math.log(2)) ** 2))


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.186s

OK


## Задание 1.3 (2 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Используйте библиотечный метод или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+1 балл**)

In [ ]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

In [6]:
def central_diff(f: Union["Function", Callable[[float], float]], h: float = 1e-6, direction='x', **named_args) -> float:
    return (
            f(**{key: value + (h if key == direction else 0) for key, value in named_args.items()}) -
            f(**{key: value - (h if key == direction else 0) for key, value in named_args.items()})
    ) / (2 * h)


def five_point_diff(f: Union["Function", Callable[[float], float]], h: float = 1e-6, direction='x', **named_args) -> float:
    return (
            -f(**{key: value - (2*h if key == direction else 0) for key, value in named_args.items()}) +
            8*f(**{key: value - (h if key == direction else 0) for key, value in named_args.items()}) -
            8*f(**{key: value + (h if key == direction else 0) for key, value in named_args.items()}) +
            f(**{key: value + (2*h if key == direction else 0) for key, value in named_args.items()})
    ) / (12 * h)


def complex_diff(f: Callable[[float], float], h: float = 1e-6, direction='x', **named_args):
    res = f(**{key: (value + h * 1j) if key == direction else value for key, value in named_args.items()})
    return res if res is math.nan else res.imag / h


## Задание 1.4 (6 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [7]:
n_children = ((0, 1, 2), (.15, .35, .55))
operators_double = (('+', '-', '*', '/', '^'), (.25, .2, .2, .2, .1))
inv_unar_prob = .5
functions = (('exp', 'log', 'sin', 'cos'), (.15, .2, .325, .325))


class Function:
    def __init__(self, depth: int, constrains=None):
        self.n_children = random.choices(n_children[0], weights=n_children[1])[0] * (not not depth)
        if self.n_children == 0:
            self.var_or_c = random.choices(vars_and_const[0], weights=vars_and_const[1])[0]
            if self.var_or_c == '$const':
                if constrains is None:
                    self.const = Dual(-5 + 10 * random.random(), 0.0)
                    return
                while True:
                    val = -5 + 10 * random.random()
                    if constrains(val):
                        break
                self.const = Dual(val, 0.0)
                return

        elif self.n_children == 1:
            self.func = random.choices(functions[0], weights=functions[1])[0]
            if self.func == 'log':
                new_constrains = [lambda a: a > 0]
            else:
                new_constrains = [None]
        elif self.n_children == 2:
            self.op = random.choices(operators_double[0], weights=operators_double[1])[0]
            if self.op == "^":
                new_constrains = [lambda a: a > 0, None]
            elif self.op == '/':
                new_constrains = [None, lambda a: a != 0]
            else:
                new_constrains = [None, None]

        if self.n_children:
            self.children = [Function(depth - 1, new_constrains[j]) for j in range(self.n_children)]

    def __call__(self, auto_diff=False, **kwargs):
        if self.n_children == 0:
            if self.var_or_c == '$const':
                if auto_diff:
                    return self.const
                else:
                    return self.const.value
            else:
                return kwargs[self.var_or_c]
        elif self.n_children == 1:
            val = self.children[0](auto_diff, **kwargs)
            if isinstance(val, float) and math.isnan(val) or isinstance(val, Dual) and math.isnan(val.value):
                return Dual(math.nan, math.nan) if auto_diff else math.nan
            match self.func:
                case 'exp':
                    if auto_diff:
                        return exp(val)
                    else:
                        return math.exp(val)
                case 'log':
                    if (
                            (isinstance(val, Dual) and val.value <= 0) or
                            (isinstance(val, float) and val <= 0) or
                            (isinstance(val, complex) and val.real <= 0)
                    ):
                        return Dual(math.nan, math.nan) if auto_diff else math.nan
                    elif auto_diff:
                        return log(val)
                    else:
                        return math.log(val)
                case 'sin':
                    if auto_diff:
                        return sin(val)
                    else:
                        return math.sin(val)
                case 'cos':
                    if auto_diff:
                        return cos(val)
                    else:
                        return math.cos(val)
        elif self.n_children == 2:
            vals = [child(auto_diff, **kwargs) for child in self.children]
            if any(
                    isinstance(val, float) and math.isnan(val) or isinstance(val, Dual) and math.isnan(val.value)
                    for val in vals
            ):
                return Dual(math.nan, math.nan) if auto_diff else math.nan
            match self.op:
                case '+':
                    return vals[0] + vals[1]
                case '-':
                    return vals[0] - vals[1]
                case '*':
                    return vals[0] * vals[1]
                case '/':
                    if abs(vals[1]) == 0:
                        return Dual(math.nan, math.nan) if auto_diff else math.nan
                    return vals[0] / vals[1]
                case '^':
                    if (
                            (isinstance(vals[0], Dual) and vals[0].value <= 0) or
                            (isinstance(vals[0], float) and vals[0] <= 0) or
                            (isinstance(vals[0], complex) and vals[0].real <= 0)
                    ):
                        return Dual(math.nan, math.nan) if auto_diff else math.nan
                    return vals[0] ** vals[1]

    def draw(self, init=True):
        if self.n_children == 0:
            if self.var_or_c == '$const':
                return f"{self.const}"
            else:
                return self.var_or_c
        if self.n_children == 1:
            return f"{self.func}({self.children[0].draw()})"

        temp = self.op.join(f"{child.draw(False)}" for child in self.children)
        if init:
            return temp
        else:
            return f"({temp})"


def diff(func: Union["Function", Callable[[Dual], Dual]]) -> Callable[[Dict[str, float]], float]:
    def dfunc_dx(**named_args: float):
        if isinstance(func, Function):
            return func(auto_diff=True, **{key: Dual(value, 1. if len(named_args) == 1 or key == named_args["direction"] else 0.) for key, value in named_args.items() if key != "direction"}).d
        else:
            return func(**{key: Dual(value, 1. if len(named_args) == 1 or key == named_args["direction"] else 0.) for key, value in named_args.items() if key != "direction"}).d

    return dfunc_dx

In [8]:
class TestAutomaticDifferential(unittest.TestCase):
    def test_random_functions(self):
        random.seed(42)
        for _ in range(10):
            f = Function(3)
            print(f.draw())
            for x in [-5 + 10 * random.random() for _ in range(10)]:
                diff1 = diff(f)(x=x)
                diff2 = complex_diff(f, x=x)
                diff3 = central_diff(f, x=x)
                assert (
                        (math.isclose(diff1, diff2) or math.isclose(diff1, diff3, rel_tol=1e-6)) or
                        (math.isnan(diff1) and math.isnan(diff2) and math.isnan(diff3))
                ), "x=%.10f, auto: %.10f, complex: %.10f, central: %.10f" % (x, diff1, diff2, diff3)


vars_and_const = (('x', '$const'), (.75, .25))


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.019s

OK


log(x*x)+sin(x)
x-((-1.215/1.185)*(x*x))
cos(sin(x))*((x/x)/cos(x))
log(x*x)
sin(x)*cos(x--2.016)
(4.469-x)/2.658
cos((x+x)-exp(x))
exp(sin(x))/(x^cos(x))
(sin(x)/x)^((4.954-x)-log(x))
exp(x+x)*(cos(x)^(-4.275-4.961))


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [9]:
class TestAutomaticDifferential(unittest.TestCase):
    def test_random_functions_grads(self):
        random.seed(42)
        for _ in range(25):
            f = Function(3)
            print(f.draw())
            for _ in range(25):
                named_args = {}
                for arg in vars_and_const[0]:
                    if arg == '$const':
                        continue
                    named_args[arg] = -3 + 6 * random.random()
                for diff_dir in named_args:
                    diff1 = diff(f)(direction=diff_dir, **named_args)
                    diff2 = complex_diff(f, direction=diff_dir, **named_args)
                    diff3 = central_diff(f, direction=diff_dir, **named_args)
                    
                    assert (
                            (math.isclose(diff1, diff2, abs_tol=1e-6) or math.isclose(diff1, diff3, rel_tol=1e-6)) or
                            (math.isnan(diff1) and math.isnan(diff2) and math.isnan(diff3))
                    ), "%s, %s, auto: %.10f, complex: %.10f, central: %.10f" % (
                        "∂f/∂%s" % diff_dir,
                        ", ".join(f"{key}={value:.10f}" for key, value in named_args.items()),
                        diff1,
                        diff2,
                        diff3
                    )


vars_and_const = (('x', 'y', 'z', '$const'), (.2, .4, .3, .1))


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.139s

OK


log(x*x)+sin(x)
x
sin(exp(z*x))
cos(y+z)+(cos(z)-(z*z))
((x+y)*y)-((y*z)-0.385)
((y/z)*(y*y))*log(z+x)
cos(z)^((x/z)+sin(y))
((y+y)*sin(x))-((z-x)*(x-z))
((y/y)-(1.577-y))-sin(y*-0.619)
(log(z)/(z+2.468))/exp(z/x)
(exp(x)/(4.673/y))*((z-z)+(x+y))
y*((y/x)+(y-x))
z
(log(z)+(y+z))/cos(z*x)
cos(x+(z+x))
y
z/(exp(y)*(y-z))
((z*z)+(-0.674*x))/x
exp(y/z)+cos(y/x)
log(cos(z)-(y*y))
(log(x)/log(z))*((z+y)*z)
y
exp(cos(cos(z)))
(z+(y*y))*((z-y)+(x+y))
cos(y-z)+z
